In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os
import random
import shutil
import time
from tqdm.notebook import tqdm
from torchinfo import summary
import pickle

In [2]:
SEED = 853

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
data_dir = 'CV/data'
images_dir = os.path.join(data_dir, 'images')
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

classes = os.listdir(images_dir)

In [4]:
pretrained_size = 388
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds= [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(pretrained_size, padding = 10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.CenterCrop(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean = pretrained_means, 
                                                std = pretrained_stds)
                       ])

In [5]:
train_data = datasets.ImageFolder(root = train_dir, 
                                  transform = train_transforms)

test_data = datasets.ImageFolder(root = test_dir, 
                                 transform = test_transforms)

In [6]:
VALID_RATIO = 0.9

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

In [7]:
valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 33202
Number of validation examples: 3690
Number of testing examples: 9225


In [9]:
BATCH_SIZE = 128

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE,
                                num_workers = 14)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE,
                                num_workers = 14)

test_iterator = data.DataLoader(test_data, 
                                batch_size = BATCH_SIZE,
                                num_workers = 14)

In [10]:
from pytorch_pretrained_vit import ViT
pretrained_model = ViT('B_16_imagenet1k', pretrained=True)

pretrained_model.fc = nn.Linear(768, len(test_data.classes))

Loaded pretrained weights.


In [11]:
for param in pretrained_model.parameters():
    param.requires_grad = False
for param in pretrained_model.fc.parameters():
    param.requires_grad = True

In [12]:
summary(pretrained_model)

Layer (type:depth-idx)                                  Param #
ViT                                                     --
├─Conv2d: 1-1                                           (590,592)
├─PositionalEmbedding1D: 1-2                            (443,136)
├─Transformer: 1-3                                      --
│    └─ModuleList: 2-1                                  --
│    │    └─Block: 3-1                                  (7,087,872)
│    │    └─Block: 3-2                                  (7,087,872)
│    │    └─Block: 3-3                                  (7,087,872)
│    │    └─Block: 3-4                                  (7,087,872)
│    │    └─Block: 3-5                                  (7,087,872)
│    │    └─Block: 3-6                                  (7,087,872)
│    │    └─Block: 3-7                                  (7,087,872)
│    │    └─Block: 3-8                                  (7,087,872)
│    │    └─Block: 3-9                                  (7,087,872)
│    │    └─Blo

In [13]:
device = torch.device('cuda')

criterion = nn.CrossEntropyLoss().to(device)

model = pretrained_model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose = True, patience=5)

In [14]:
EPOCHS = 500
STEPS_PER_EPOCH = len(train_iterator)
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH

MAX_LRS = [p['lr'] for p in optimizer.param_groups]

In [15]:
def calculate_topk_accuracy(y_pred, y, k = 1):
    with torch.no_grad():
        batch_size = y.shape[0]
        _, top_pred = y_pred.topk(k, 1)
        top_pred = top_pred.t()
        correct = top_pred.eq(y.view(1, -1).expand_as(top_pred))
        correct_1 = correct[:1].reshape(-1).float().sum(0, keepdim = True)
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim = True)
        acc_1 = correct_1 / batch_size
        acc_k = correct_k / batch_size
    return acc_1, acc_k

In [16]:
def train(model, iterator, optimizer, criterion, scheduler, device):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.train()
    
    for (x, y) in tqdm(iterator):
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc_1 += acc_1.item()
        epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [17]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
            epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)
    epoch_acc_5 /= len(iterator)
        
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
#torch.save(model.state_dict(), '/scratch/dpj7913/CV/vit/best-model_at_' + str(epoch) + '.pth')

In [20]:
model.load_state_dict(torch.load('/scratch/dpj7913/CV/vit/best-model_at_4.pth'))

<All keys matched successfully>

In [ ]:
torch.cuda.empty_cache()

best_valid_loss = float('inf')
accuracies = {}

for epoch in tqdm(range(5, EPOCHS)):
    
    start_time = time.monotonic()
    
    train_loss, train_acc_1, train_acc_5 = train(model, train_iterator, optimizer, criterion, scheduler, device)
    valid_loss, valid_acc_1, valid_acc_5 = evaluate(model, valid_iterator, criterion, device)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/scratch/dpj7913/CV/vit/best-model_at_' + str(epoch) + '.pth')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc @1: {train_acc_1*100:6.2f}% | ' \
          f'Train Acc @5: {train_acc_5*100:6.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc @1: {valid_acc_1*100:6.2f}% | ' \
          f'Valid Acc @5: {valid_acc_5*100:6.2f}%')
    
    scheduler.step(valid_loss)
    accuracies[epoch] = {'train': train_acc_1, 'val': valid_acc_1}
    
    if epoch%50 == 0:
        f = open("accuracies_vit.pkl", "wb")
        pickle.dump(accuracies, f)
        f.close()

  0%|          | 0/495 [00:00<?, ?it/s]

  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 06 | Epoch Time: 9m 46s
	Train Loss: 0.746 | Train Acc @1:  66.89% | Train Acc @5:  66.89%
	Valid Loss: 0.790 | Valid Acc @1:  63.80% | Valid Acc @5:  63.80%


  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 07 | Epoch Time: 9m 47s
	Train Loss: 0.746 | Train Acc @1:  67.03% | Train Acc @5:  67.03%
	Valid Loss: 0.789 | Valid Acc @1:  64.49% | Valid Acc @5:  64.49%


...

  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 47 | Epoch Time: 9m 47s
	Train Loss: 0.725 | Train Acc @1:  68.35% | Train Acc @5:  68.35%
	Valid Loss: 0.780 | Valid Acc @1:  65.04% | Valid Acc @5:  65.04%


  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 48 | Epoch Time: 9m 47s
	Train Loss: 0.726 | Train Acc @1:  68.27% | Train Acc @5:  68.27%
	Valid Loss: 0.780 | Valid Acc @1:  65.04% | Valid Acc @5:  65.04%


  0%|          | 0/260 [00:00<?, ?it/s]